# Classification with gpt-4-vision-preview
## Overview

This notebook is based on a Notebook by Michael Achmann form the social-media-lab project. Minor changes were made to deal with data loss, TOS problems with some images used for annotation and network reliabilty problems.

### Project Information

- Project Website: [social-media-lab.net](https://social-media-lab.net/)
- GitHub Repository: [https://github.com/michaelachmann/social-media-lab](https://github.com/michaelachmann/social-media-lab)

## License Information

This notebook, along with all other notebooks in the project, is licensed under the following terms:

- License: [GNU General Public License version 3.0 (GPL-3.0)](https://www.gnu.org/licenses/gpl-3.0.de.html)
- License File: [LICENSE.md](https://github.com/michaelachmann/social-media-lab/blob/main/LICENSE.md)


## Citation


```
Michael Achmann. (2024). michaelachmann/social-media-lab: 2024-1-22 (v0.0.10). Zenodo. https://doi.org/10.5281/zenodo.8199901
```

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip -q install openai backoff --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.5/257.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00


# Preparing Data

In [ ]:
# importing the zipfile module
from zipfile import ZipFile

# loading the temp.zip and creating a zip object
with ZipFile("/content/drive/MyDrive/Uni/SocialMedSeminar/Daten/TikTok_DATA_COMPLETE-20240312T085053Z-001.zip", 'r') as zObject:

	# Extracting all the members of the zip
	# into a specific location.
	zObject.extractall(
		path="/content/drive/MyDrive/Uni/SocialMedSeminar/Daten/tiktok_final")


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Uni/SocialMedSeminar/Daten/insta_final/INSTA_DATA_COMPLETE/InstaAll_EN_DE_merged.csv')

In [ ]:
# für instagram
df['image_path'] = df.apply(lambda row: f"/content/drive/MyDrive/Uni/SocialMedSeminar/Daten/insta_final/INSTA_DATA_COMPLETE/{row['Hashtag']}_images_names/{row['Hashtag']}_{row['Filename']}.jpg", axis=1)

In [ ]:
# für tiktok
df['image_path'] = df.apply(lambda row: f"/content/drive/MyDrive/Uni/SocialMedSeminar/Daten/tiktok_final/TikTok_DATA_COMPLETE/TikTok Bilder gemerged/{row['Hashtag']}/{row['Hashtag']}/{row['id']}.jpeg", axis=1)

In [ ]:
df

,Unnamed: 0,Filename,Account,User Name,Followers at Posting,Post Created,Post Created Date,Post Created Time,Type,Total Interactions,...,Title,Description,Image Text,Sponsor Id,Sponsor Name,Total Interactions (weighted — Likes 1x Comments 1x ),Overperforming Score,language,Hashtag,Sentiment
0,0,1,NASA Earth,nasaearth,1725776.0,2023-01-19 23:33:05 CET,19.01.23,23:33:05,Photo,"1,500,579",...,NaN,Not your typical ice cube. 🧊\n\nAlthough it is...,NaN,NaN,NaN,"1,500,579",98.01,Language.ENGLISH,climatechange,neutral
1,1,2,NASA,nasa,87161799.0,2023-01-19 23:33:05 CET,19.01.23,23:33:05,Photo,"1,495,285",...,NaN,Not your typical ice cube. 🧊\n\nAlthough it is...,NaN,NaN,NaN,"1,495,285",Jan 13,Language.ENGLISH,climatechange,neutral
2,2,3,Pubity,pubity,35467940.0,2023-12-10 08:06:05 CET,10.12.23,08:06:05,Photo,"477,81",...,NaN,2023 will be the hottest year recorded in hist...,2023 WILL OFFICIALLY BE THE HOTTEST YEAR RECOR...,NaN,NaN,"477,81",01. Sep,Language.ENGLISH,climatechange,positive
3,3,4,MEMEZAR • MEMES & MEME-CULTURE,memezar,22449577.0,2023-11-15 13:38:40 CET,15.11.23,13:38:40,Photo,"446,001",...,NaN,I just know someone would try drinking them to...,Scientists create Liquid Trees; a tank full of...,NaN,NaN,"446,001",Jan 24,Language.ENGLISH,climatechange,neutral
4,4,5,Pubity,pubity,33392362.0,2023-07-15 16:48:32 CEST,15.07.23,16:48:32,Photo,"439,535",...,NaN,"According to one professor, we can stop global...",平節 P PROFESSOR SUGGESTS USING 139B GALLONS OF ...,NaN,NaN,"439,535",-1.1,Language.ENGLISH,climatechange,positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3797,3797,696,Snap_Sri _Lanka,snap.sri_lanka,NaN,2023-08-10 20:03:23 CEST,10.08.23,20:03:23,Album,"3,891",...,NaN,Posted @withregram • @jellybeesl 🖇️\nUnapologe...,NaN,NaN,NaN,"3,891",0,Language.ENGLISH,savetheplanet,neutral
3798,3798,697,The Undisputable Vegans,the.undisputable.vegans,83955.0,2023-05-29 07:48:13 CEST,29.05.23,07:48:13,Photo,"3,877",...,NaN,This art by @veganactionart / @skorskamariia r...,NaN,NaN,NaN,"3,877",Mar 55,Language.ENGLISH,savetheplanet,negative
3799,3799,698,The Undisputable Vegans,the.undisputable.vegans,77214.0,2023-04-25 04:02:34 CEST,25.04.23,04:02:34,Photo,"3,87",...,NaN,Talking about animal rights with vegans vs tal...,Talking about animal rights with vegans VS Tal...,NaN,NaN,"3,87",Mar 65,Language.ENGLISH,savetheplanet,neutral
3800,3800,699,Kluchit,kluchit,153968.0,2023-07-28 12:34:02 CEST,28.07.23,12:34:02,Photo,"3,87",...,NaN,The era of global warming has ended and “the e...,"World News Global warming is over, the era of ...",NaN,NaN,"3,87",Mar 14,Language.ENGLISH,savetheplanet,negative


# Preparing Prompts
These are main prompts used during testing. All three prompts had iterative minor improvements done to them to improve outcomes. These minor changes were not tracked.

Prompt 3 is significantly shorter, as it was part of an experiment to replace part of the gpt-4 annotation with CLIP. Results from this experiment were not usable.

In [ ]:
# Version 1
prompt = """
You are an AI assistant with years of training in image analysis for social media. Use the following annotation manual to code the provided image. We're **not** interested in the identity of any person in the image, please anonymize any personal information and concentrate on the objective image analysis framework outlined below.
**Objective**: Perform a content analysis of a given Instagram post, which is possibly related to climate change, because it has been posted with one of the following hashtags: climatechange, savetheplanet, climatecrisis, klimawandel, klimaschutz or klimakrise. The posts are in english or german.
### Annotation Guide
#### 1. **Correlation to the topic** (Boolean)
   - True / False (This category determines if the given picture has a correlation with a climate hashtag. We chose the six following hashtags: #Klimawandel, #Klimakrise,# Klimaschutz,#savetheplanet, #climatekrisis, #climatechange. As some pictures are tagged with climate hashtags but have no obvious correlation with them it is important for us to take this into account.)
#### 2. **Type of media content**(Categorical)
   - Text / Photo / Drawing / Diagram (Multiple categories at the same time are possible)
#### 3. **Impression or Sentiment** (Categorical)
   - Positive, Neutral, Negative (This category determines the impression/sentiment towards an image. Images can convey positive, negative or neutral sentiment. Choose the sentiment that is intended to be expressed with the image. )
#### 4. **Animals** (Boolean)
   - True / False (Various climate posts depict different wildlife and animals. Animals can be depicted in photographs, but also illustrated in a drawing. They can also appear alongside humans. Animal emojis also count as animals but if an animal appears in a logo it does not count as an animal. Choose yes if you can see an animal.)
#### 5. **Nature** (Boolean)
   - True / False (Various climate posts depict nature in different forms, either with animals, humans and artifacts or just by itself. Sometimes nature is also embedded in an urban environment or is depicted in a drawing. If there are more than two houseplants in the image it also counts as nature. Maps and depiction of earth as a plant does not count as nature.)
#### 6. **People** (Boolean)
   - True / False (Many climate posts show humans doing different activities. People are depicted in photos, drawings and other forms of posts (e.g. in diagrams). If people are only depicted partly it also counts as people. Choose yes if you can see people in a post.)
#### 7. **Natural Disasters** (Boolean)
   - True / False (Any type of destruction of nature and natural disasters that destroy the environment and appear in different forms. From littering and polluting the planet to floods, wildfires and earthquakes, check the box if there is visible damage to the environment or to human settlements and infrastructure. These events are triggered by humans or natural forces and can take various forms. They can be presented in pictures and also in drawings. Choose yes if you can see a natural disaster or the destruction of nature)
#### 8. **Artifacts ** (Boolean)
   - True / False (Man-made artifacts are often depicted in posts. Things like cars, houses, phones, bottles, tables and so on. Things that people are wearing like clothes, piercings and glasses do not count as artifacts. If artifacts are in the background of an image but not relevant to the post (e.g. a person is posting a selfie, is standing in their room and a TV is in the background), choose no. However, if the TV is relevant to the post (e.g. the TV is on and is in the focus of the image or the TV is in an unsusal space like the woods), choose yes. )
#### 9. **Education, Facts or News** (Boolean)
   - True / False (Posts can also contain educational information, calls to actions, instructions and tips to save the planet, facts (true or false) about the topic, news and other types of information. Even if the given content is not correct the intention of the post is important. Questions also count as educational as you can expect them to be answered in the course of the video (if it is a frame of a video) or in the description.)
**Analytical Process**
  - For each Instagram post, identify and record occurrences of the items listed in the provided table.
  - Categorize findings under the appropriate theory and visual frame.
**Reporting**: Summarize the findings in a structured JSON format based on the variable names in the Annotation guide. Respond only in JSON, respect the data types indicated in the manual.
"""

In [ ]:
# Version 2 - Used Version
prompt = """
You are an AI assistant with years of training in image analysis for social media. Use the following annotation manual to code the provided image. We're **not** interested in the identity of any person in the image, please anonymize any personal information and concentrate on the objective image analysis framework outlined below.
**Objective**: Perform a content analysis of a given Social Media post, posted with one of the following hashtags: climatechange, savetheplanet, climatecrisis, klimawandel, klimaschutz or klimakrise. The posts are in english or german.
### Annotation Guide
#### 1. **Correlation to the topic** (Boolean)
   - True / False (This category determines if the given picture has a correlation with a climate hashtag.)
#### 2. **Type of media content**(Categorical)
   - Text / Photo / Drawing / Diagram (This category determines what kind of media content is depicted. Emojis count as drawings if they appear without a text, otherwise they are part of a text.)
#### 3. **Impression or Sentiment** (Categorical)
   - Positive, Neutral, Negative (This category determines the impression/sentiment towards an image. Choose the sentiment that is intended to be expressed with the image. )
#### 4. **Animals** (Boolean)
   - True / False (Animals can be depicted in photographs, but also illustrated in a drawing. They can also appear alongside humans. Animal emojis also count as animals but if an animal appears in a logo it does not count as an animal.)
#### 5. **Nature** (Boolean)
   - True / False (Various climate posts depict nature in different forms, either with animals, humans and artifacts or just by itself. Sometimes nature is also embedded in an urban environment or is depicted in a drawing. If there are more than two houseplants in the image it also counts as nature. Maps and depiction of earth as a plant does not count as nature.)
#### 6. **People** (Boolean)
   - True / False (Choose true if you can see people in a post.)
#### 7. **Natural Disasters** (Boolean)
   - True / False (Any type of destruction of nature and natural disasters.)
#### 8. **Artifacts ** (Boolean)
   - True / False (Things like cars, houses, phones, bottles, tables and so on. Things that people are wearing like clothes, piercings and glasses do not count as artifacts. If artifacts are in the background of an image but not relevant to the post (e.g. a person is posting a selfie, is standing in their room and a TV is in the background), choose no. However, if the TV is relevant to the post, choose yes. )
#### 9. **Education, Facts or News** (Boolean)
   - True / False (Questions also count as educational.)
**Analytical Process**
  - For each Instagram post, identify and record occurrences of the items or sceneries listed in the provided table.
**Reporting**: Summarize the findings in a structured JSON format based on the variable names in the Annotation guide. Respond only in JSON, respect the data types indicated in the manual.
"""

In [ ]:
# Version 3 - remaining annotations were tried in CLIP
prompt = """
You are an AI assistant with years of training in image analysis for social media. Use the following annotation manual to code the provided image. We're **not** interested in the identity of any person in the image, please anonymize any personal information and concentrate on the objective image analysis framework outlined below.
**Objective**: Perform a content analysis of a given Social Media post, posted with one of the following hashtags: climatechange, savetheplanet, climatecrisis, klimawandel, klimaschutz or klimakrise. The posts are in english or german.
### Annotation Guide
#### 1. **Correlation to the topic** (Boolean)
   - True / False (This category determines if the given picture has a correlation with a climate hashtag.)
#### 2. **Type of media content**(Categorical)
   - Text / Photo / Drawing / Diagram (This category determines what kind of media content is depicted. Emojis count as drawings if they appear without a text, otherwise they are part of a text. If the picture does not clearly belong to one category add a second variable Type of media content 2 and add the category there.)
#### 3. **Impression or Sentiment** (Categorical)
   - Positive, Neutral, Negative (This category determines the impression/sentiment towards an image. Choose the sentiment that is intended to be expressed with the image. )
**Analytical Process**
  - For each Instagram post, identify and record occurrences of the items or sceneries listed in the provided table.
**Reporting**: Summarize the findings in a structured JSON format based on the variable names in the Annotation guide. Respond only in JSON, respect the data types indicated in the manual.
"""

In [ ]:
import pandas as pd
import json
import re
from pandas.io.json import json_normalize

def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

def parse_response(response, identifier):
    try:
        if isinstance(response, str):
            response = json.loads(response)
        response = flatten_json(response)
        response['ID'] = identifier
        return response
    except json.JSONDecodeError:
        match = re.search(r'```json\n([\s\S]+)\n```', response)
        if match:
            try:
                json_data = json.loads(match.group(1))
                json_data = flatten_json(json_data)
                json_data['image_path'] = identifier
                return json_data
            except json.JSONDecodeError:
                pass
        return {'image_path': identifier, 'error': response}

# Annotation

Our 12 datasets were split at the hashtag do minimize data loss.

Average Dataset consisted of 550 images.

The annotated data was immediately saved to a csv.

In [ ]:
import base64
from tqdm.notebook import tqdm
import openai
from google.colab import userdata
import pandas as pd
import backoff

# Retrieving OpenAI API Key
api_key = userdata.get('openai-lehrstuhl-api')

# Initialize OpenAI client
client = openai.OpenAI(api_key=api_key, organization="org-0KdP9Uf8i0yybiV0am6DyaIZ")

# Cost per token
prompt_cost = 0.01 / 1000  # Cost per prompt token
completion_cost = 0.03 / 1000  # Cost per completion token

# Initialize total cost
total_cost = 0.0

def encode_image(image_path):
    """
    Encodes an image to base64.

    :param image_path: Path to the image file.
    :return: Base64 encoded string of the image.
    """
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

@backoff.on_exception(backoff.expo, (openai.RateLimitError, openai.APIError))
def run_request(prompt, base64_image):
    """
    Sends a request to OpenAI with given prompt and image.

    :param prompt: Text prompt for the request.
    :param base64_image: Base64 encoded image.
    :return: Response from the API.
    """
    messages = [{
        "role": "user",
        "content": [
            {"type": "text", "text": prompt},
            {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}", "detail":"low"}}
        ]
    }]

    return client.chat.completions.create(
        model="gpt-4-vision-preview",
        temperature=0,
        messages=messages,
        max_tokens=600
    )

responses = []
data = []

sample_df = df[df['Hashtag'] == "climatechange"]

#sample_df = sample_df.sample(5)


for index, row in tqdm(sample_df.iterrows(), total=sample_df.shape[0]):
        try:
            image_path = row['image_path']
            base64_image = encode_image(image_path)
            response = run_request(prompt, base64_image)
            r = response.choices[0].message.content

            current_prompt_cost = response.usage.prompt_tokens * prompt_cost
            current_completion_cost = response.usage.completion_tokens * completion_cost
            current_cost = current_prompt_cost + current_completion_cost
            total_cost += current_cost

            print(f"This round cost ${current_cost:.6f}")

            responses.append({'image_path': row['image_path'], 'classification': r})

            processed_data = parse_response(r, row['image_path'])
            data.append(processed_data)

        except Exception as e:
            print(f"Error processing image {row['image_path']}: {e}")


print(f"Total cost ${total_cost:.6f}")
data_df = pd.DataFrame(data)
total_df = pd.merge(df, data_df, how="left", on="image_path")
total_df.to_csv('/content/drive/MyDrive/Uni/SocialMedSeminar/Daten/tiktok_gpt_annotiert_climatechange.csv')

  0%|          | 0/480 [00:00<?, ?it/s]

This round cost $0.010010
This round cost $0.010010
This round cost $0.010010
This round cost $0.010010
This round cost $0.010010
This round cost $0.010010
This round cost $0.010010
This round cost $0.010010
This round cost $0.010010
This round cost $0.010010
This round cost $0.010010
This round cost $0.010010
This round cost $0.010010
This round cost $0.010010
This round cost $0.010010
This round cost $0.010010
This round cost $0.010010
This round cost $0.010010
This round cost $0.010010
This round cost $0.010010
This round cost $0.010010
This round cost $0.010010
This round cost $0.010010
This round cost $0.010010
This round cost $0.010010
This round cost $0.010010
This round cost $0.010010
This round cost $0.010010
This round cost $0.010010
This round cost $0.010010
This round cost $0.010010
This round cost $0.010010
This round cost $0.010010
This round cost $0.010010
This round cost $0.010010
This round cost $0.010010
This round cost $0.010010
This round cost $0.010010
This round c

In [ ]:
data_df = pd.DataFrame(data)

In [ ]:
sample_df

In [ ]:
data_df

,Correlation_to_the_topic,Type_of_media_content,Impression_or_Sentiment,Animals,Nature,People,Natural_Disasters,Artifacts,Education_Facts_or_News,image_path
0,True,Photo,Neutral,False,False,True,False,False,False,/content/drive/MyDrive/Uni/SocialMedSeminar/Da...
1,True,Text,Neutral,False,False,True,False,False,True,/content/drive/MyDrive/Uni/SocialMedSeminar/Da...
2,False,Photo,Neutral,False,False,True,False,True,False,/content/drive/MyDrive/Uni/SocialMedSeminar/Da...
3,False,Photo,Neutral,False,False,True,False,True,True,/content/drive/MyDrive/Uni/SocialMedSeminar/Da...
4,False,Photo,Neutral,False,False,True,False,True,False,/content/drive/MyDrive/Uni/SocialMedSeminar/Da...
...,...,...,...,...,...,...,...,...,...,...
594,True,Photo,Neutral,False,False,True,False,True,True,/content/drive/MyDrive/Uni/SocialMedSeminar/Da...
595,True,Photo,Negative,False,False,False,False,True,True,/content/drive/MyDrive/Uni/SocialMedSeminar/Da...
596,True,Photo,Neutral,False,True,True,False,False,False,/content/drive/MyDrive/Uni/SocialMedSeminar/Da...
597,True,Photo,Neutral,False,False,True,False,False,True,/content/drive/MyDrive/Uni/SocialMedSeminar/Da...


#   Evaluating results for prompt engineering

In [ ]:
import pandas as pd
from IPython.display import display, Image
import random

# Assuming your DataFrame is already loaded and named data_df
# data_df = pd.read_csv('your_data_file.csv') # Uncomment if you need to load the DataFrame

def display_random_image_and_classification(df):
    # Select a random row from the DataFrame
    random_row = df.sample(1).iloc[0]

    # Get the image path and classification from the row
    image_path = random_row['image_path'] # Replace 'image_path' with the actual column name

    # Display the image
    display(Image(filename=image_path))

    # Display the classification
    print(f"Correlation_to_the_topic: {random_row['Correlation_to_the_topic']}")
    print(f"Type_of_media_content: {random_row['Type_of_media_content']}")
    print(f"Impression_or_Sentiment: {random_row['Impression_or_Sentiment']}")
    print(f"Animals: {random_row['Animals']}")
    print(f"Nature: {random_row['Nature']}")
    print(f"People: {random_row['People']}")
    print(f"Natural_Disasters: {random_row['Natural_Disasters']}")
    print(f"Artifacts: {random_row['Artifacts']}")
    print(f"Education_Facts_or_News: {random_row['Education_Facts_or_News']}")

# Call the function to display an image and its classification
display_random_image_and_classification(data_df)


In [ ]:
display_random_image_and_classification(data_df)

In [ ]:
total_df = pd.merge(df, data_df, how="left", on="image_path")

In [ ]:
total_df[~pd.isna(total_df['Correlation_to_the_topic'])].head()

,Unnamed: 0,ID,Time of Posting,Type of Content,video_url,image_url,Username,Video Length (s),Expiration,Caption,Is Verified,Stickers,Accessibility Caption,Attribution URL,image_path,Performance,Environment_Environment,Environment_Location,DressStyle_DressStyle,DressStyle_RolledUpSleeves
44,44,2638549336000085388_1484534097,2021-08-12 09:13:30,Video,NaN,NaN,afd.bund,5.000,2021-08-13 09:13:30,NaN,True,[],NaN,https://www.threads.net/t/CSeAZYrotni,/content/media/images/afd.bund/263854933600008...,CampaignOrPartyEvent,Outdoors,StreetOrPlaza,Formal,False
70,70,2643951511915139810_1484534097,2021-08-19 20:06:41,Image,NaN,NaN,afd.bund,NaN,2021-08-20 20:06:41,NaN,True,[],Photo by Alternative für Deutschland on August...,https://www.threads.net/t/CSxK2ybDz9D,/content/media/images/afd.bund/264395151191513...,CampaignOrPartyEvent,Indoors,EventHall,Formal,False
93,93,2645978622294450871_1484534097,2021-08-22 15:14:11,Video,NaN,NaN,afd.bund,2.066,2021-08-23 15:14:11,NaN,True,[],NaN,NaN,/content/media/images/afd.bund/264597862229445...,CampaignOrPartyEvent,Outdoors,StreetOrPlaza,Formal,False
162,162,2665468139419335791_1484534097,2021-09-18 12:36:23,Image,NaN,NaN,afd.bund,NaN,2021-09-19 12:36:23,NaN,True,[],Photo by Alternative für Deutschland on Septem...,https://www.threads.net/t/CT9o2o6NZLg,/content/media/images/afd.bund/266546813941933...,ProtestEvent,Outdoors,StreetOrPlaza,Casual,False
165,165,2671444449736006853_1484534097,2021-09-26 18:30:15,Image,NaN,NaN,afd.bund,NaN,2021-09-27 18:30:15,NaN,True,"[{'height': 0.044419695058272, 'rotation': 0, ...",Photo by Alternative für Deutschland on Septem...,NaN,/content/media/images/afd.bund/267144444973600...,MediaEvent,Indoors,TVStudio,Formal,False


In [ ]:
total_df.to_csv('/content/drive/MyDrive/Uni/SocialMedSeminar/Daten/insta_gpt_annotiert_savetheplanet.csv')

### Processing output csv's to better fit data structure


In [ ]:
# importing insta data
df1 = pd.read_csv('/content/drive/MyDrive/Uni/SocialMedSeminar/Daten/insta_gpt_annotiert_climatechange.csv')
df2 = pd.read_csv('/content/drive/MyDrive/Uni/SocialMedSeminar/Daten/insta_gpt_annotiert_climatecrisis.csv')
df3 = pd.read_csv('/content/drive/MyDrive/Uni/SocialMedSeminar/Daten/insta_gpt_annotiert_klimakrise.csv')
df4 = pd.read_csv('/content/drive/MyDrive/Uni/SocialMedSeminar/Daten/insta_gpt_annotiert_klimaschutz.csv')
df5 = pd.read_csv('/content/drive/MyDrive/Uni/SocialMedSeminar/Daten/insta_gpt_annotiert_klimawandel.csv')
df6 = pd.read_csv('/content/drive/MyDrive/Uni/SocialMedSeminar/Daten/insta_gpt_annotiert_savetheplanet.csv')

In [ ]:
# importing tiktok data
df11 = pd.read_csv('/content/drive/MyDrive/Uni/SocialMedSeminar/Daten/tiktok_gpt_annotiert_climatechange.csv')
df22 = pd.read_csv('/content/drive/MyDrive/Uni/SocialMedSeminar/Daten/tiktok_gpt_annotiert_climatecrisis.csv')
df33 = pd.read_csv('/content/drive/MyDrive/Uni/SocialMedSeminar/Daten/tiktok_gpt_annotiert_klimakrise.csv')
df44 = pd.read_csv('/content/drive/MyDrive/Uni/SocialMedSeminar/Daten/tiktok_gpt_annotiert_klimaschutz.csv')
df55 = pd.read_csv('/content/drive/MyDrive/Uni/SocialMedSeminar/Daten/tiktok_gpt_annotiert_klimawandel.csv')
df66 = pd.read_csv('/content/drive/MyDrive/Uni/SocialMedSeminar/Daten/tiktok_gpt_annotiert_savetheplanet.csv')

In [ ]:
# checking
df66


,Unnamed: 0.1,Unnamed: 0,id,thread_id,author,author_full,author_followers,author_likes,author_videos,author_avatar,...,Nature,People,Natural_Disasters,Artifacts,Education_Facts_or_News,Correlation to the topic,Type of media content,Impression or Sentiment,Natural Disasters,"Education, Facts or News"
0,0,0,7172368921318657322,7172368921318657322,creativesociety.official,Creative Society project,226500,1555,1060,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,7256810993311812907,7256810993311812907,wffnews,WFF News,148500,4724,181,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,7224978138751618305,7224978138751618305,thethinkingreatape,TheThinkingreatApe,32400,15,337,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,7291671021827116334,7291671021827116334,just_breathe_sunshine,Just_breathe,52,37,22,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,7252904683222895918,7252904683222895918,cbsnews,cbsnews,3600000,633,1669,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3098,3098,3098,7116909805238848773,7116909805238848773,holdcrofthandmades,Evie | Holdcroft Handmades,3540,2612,102,NaN,...,False,True,False,True,True,NaN,NaN,NaN,NaN,NaN
3099,3099,3099,7034644508419820846,7034644508419820846,rewildcarlyrose,carly rose 🌹,82200,318,168,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3100,3100,3100,7126465295186742534,7126465295186742534,joo_su,Josu 🌎,1600000,22600,793,NaN,...,True,True,False,True,False,NaN,NaN,NaN,NaN,NaN
3101,3101,3101,6877839838687595777,6877839838687595777,angryearth,ANGRY EARTH,852000,152,250,NaN,...,False,False,False,False,True,NaN,NaN,NaN,NaN,NaN


In [ ]:
# dropping unneeded rows from df
df11_filtered = df11[df11['Hashtag'] == "climatechange"]
df22_filtered = df22[df22['Hashtag'] == "climatecrisis"]
df33_filtered = df33[df33['Hashtag'] == "klimakrise"]
df44_filtered = df44[df44['Hashtag'] == "klimaschutz"]
df55_filtered = df55[df55['Hashtag'] == "klimawandel"]
df66_filtered = df66[df66['Hashtag'] == "savetheplanet"]

In [ ]:
# checking
df55_filtered.head()
print(df55.columns)
print(df66.columns)

Index(['Unnamed: 0.1', 'Unnamed: 0', 'id', 'thread_id', 'author',
       'author_full', 'author_followers', 'author_likes', 'author_videos',
       'author_avatar', 'body', 'timestamp', 'unix_timestamp', 'is_duet',
       'is_ad', 'music_name', 'music_id', 'music_url', 'music_thumbnail',
       'music_author', 'video_url', 'tiktok_url', 'thumbnail_url', 'likes',
       'comments', 'shares', 'plays', 'hashtags', 'challenges',
       'diversification_labels', 'location_created', 'stickers', 'effects',
       'warning', 'language', 'Hashtag', 'identifier', 'Sentiment',
       'image_path', 'Correlation_to_the_topic', 'Type_of_media_content',
       'Impression_or_Sentiment', 'Animals', 'Nature', 'People',
       'Natural_Disasters', 'Artifacts', 'Education_Facts_or_News',
       'Correlation to the topic', 'Type of media content',
       'Impression or Sentiment', 'Natural Disasters',
       'Education, Facts or News'],
      dtype='object')
Index(['Unnamed: 0.1', 'Unnamed: 0', 'id', 'thr

In [ ]:
# fixing df55 and 66
to_keep = ['Unnamed: 0.1', 'Unnamed: 0', 'id', 'thread_id', 'author',
       'author_full', 'author_followers', 'author_likes', 'author_videos',
       'author_avatar', 'body', 'timestamp', 'unix_timestamp', 'is_duet',
       'is_ad', 'music_name', 'music_id', 'music_url', 'music_thumbnail',
       'music_author', 'video_url', 'tiktok_url', 'thumbnail_url', 'likes',
       'comments', 'shares', 'plays', 'hashtags', 'challenges',
       'diversification_labels', 'location_created', 'stickers', 'effects',
       'warning', 'language', 'Hashtag', 'identifier', 'Sentiment',
       'image_path', 'Correlation_to_the_topic', 'Type_of_media_content',
       'Impression_or_Sentiment', 'Animals', 'Nature', 'People',
       'Natural_Disasters', 'Artifacts', 'Education_Facts_or_News']
df55_filtered_for_real = df55_filtered[to_keep]
df66_filtered_for_real = df66_filtered[to_keep]



,Unnamed: 0.1,Unnamed: 0,id,thread_id,author,author_full,author_followers,author_likes,author_videos,author_avatar,...,image_path,Correlation_to_the_topic,Type_of_media_content,Impression_or_Sentiment,Animals,Nature,People,Natural_Disasters,Artifacts,Education_Facts_or_News
2777,2777,2777,7275702370510638342,7275702370510638342,touya.t,nobody,27900,7061,94,NaN,...,/content/drive/MyDrive/Uni/SocialMedSeminar/Da...,True,Photo,Neutral,False,True,False,False,False,False
2778,2778,2778,7303910870201978144,7303910870201978144,beesteez,BEESTEEZ,1400000,673,346,NaN,...,/content/drive/MyDrive/Uni/SocialMedSeminar/Da...,False,Photo,Neutral,False,True,True,False,False,False
2779,2779,2779,7098808229173464325,7098808229173464325,letourplanetlive,🌍🌱,54300,1155,198,NaN,...,/content/drive/MyDrive/Uni/SocialMedSeminar/Da...,True,Text,Neutral,False,False,True,False,False,True
2780,2780,2780,7087012874148744475,7087012874148744475,kuys_bobby,Kuys_Bob🔘,225400,41500,877,NaN,...,/content/drive/MyDrive/Uni/SocialMedSeminar/Da...,False,Photo,Neutral,False,False,False,False,True,False
2781,2781,2781,7141304046345866502,7141304046345866502,angryearth,ANGRY EARTH,853700,158,252,NaN,...,/content/drive/MyDrive/Uni/SocialMedSeminar/Da...,True,Drawing,Negative,True,False,False,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3098,3098,3098,7116909805238848773,7116909805238848773,holdcrofthandmades,Evie | Holdcroft Handmades,3540,2612,102,NaN,...,/content/drive/MyDrive/Uni/SocialMedSeminar/Da...,False,Photo,Positive,False,False,True,False,True,True
3099,3099,3099,7034644508419820846,7034644508419820846,rewildcarlyrose,carly rose 🌹,82200,318,168,NaN,...,/content/drive/MyDrive/Uni/SocialMedSeminar/Da...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3100,3100,3100,7126465295186742534,7126465295186742534,joo_su,Josu 🌎,1600000,22600,793,NaN,...,/content/drive/MyDrive/Uni/SocialMedSeminar/Da...,True,Photo,Neutral,False,True,True,False,True,False
3101,3101,3101,6877839838687595777,6877839838687595777,angryearth,ANGRY EARTH,852000,152,250,NaN,...,/content/drive/MyDrive/Uni/SocialMedSeminar/Da...,True,Drawing,Negative,True,False,False,False,False,True


In [ ]:
df55_filtered_for_real

,Unnamed: 0.1,Unnamed: 0,id,thread_id,author,author_full,author_followers,author_likes,author_videos,author_avatar,...,image_path,Correlation_to_the_topic,Type_of_media_content,Impression_or_Sentiment,Animals,Nature,People,Natural_Disasters,Artifacts,Education_Facts_or_News
2157,2157,2157,7280204529113615648,7280204529113615648,politik_zitate_mehr,zitate_granate,13900,43,81,NaN,...,/content/drive/MyDrive/Uni/SocialMedSeminar/Da...,False,Photo,Neutral,False,False,True,False,False,False
2158,2158,2158,7300674878783360289,7300674878783360289,hoss_.hopf_,Hoss_.Hopf_,20900,12,156,NaN,...,/content/drive/MyDrive/Uni/SocialMedSeminar/Da...,True,Photo,Neutral,False,False,True,False,False,True
2159,2159,2159,7323077394514906401,7323077394514906401,mammutfrisstnazis,linksgrünversifftesMammut,1519,812,161,NaN,...,/content/drive/MyDrive/Uni/SocialMedSeminar/Da...,False,Video,Neutral,False,False,True,False,False,True
2160,2160,2160,6862733837055659270,6862733837055659270,robbubble,RobBubble,58800,1567,77,NaN,...,/content/drive/MyDrive/Uni/SocialMedSeminar/Da...,True,Text,Neutral,False,False,True,False,False,True
2161,2161,2161,7323084190763781409,7323084190763781409,gglueckskeks2112,glueckskeks2112,1892,96100,414,NaN,...,/content/drive/MyDrive/Uni/SocialMedSeminar/Da...,True,Photo,Negative,False,True,False,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2772,2772,2772,7086125814005255430,7086125814005255430,heeyleonie,Leonie Schöler,173700,20400,473,NaN,...,/content/drive/MyDrive/Uni/SocialMedSeminar/Da...,True,Photo,Neutral,False,True,True,False,False,True
2773,2773,2773,7210319548384283909,7210319548384283909,neumeiersmoritz,Moritz Neumeier,8509,0,21,NaN,...,/content/drive/MyDrive/Uni/SocialMedSeminar/Da...,True,Photo,Negative,False,False,True,False,True,False
2774,2774,2774,6981510960070610182,6981510960070610182,dr.dubbert,Philipp Dubbert,228900,3783,402,NaN,...,/content/drive/MyDrive/Uni/SocialMedSeminar/Da...,False,Photo,Neutral,False,True,True,False,False,False
2775,2775,2775,7011161118722854150,7011161118722854150,helgemark,Helge Mark,1200000,6952,623,NaN,...,/content/drive/MyDrive/Uni/SocialMedSeminar/Da...,True,Photo,Neutral,False,False,True,False,False,True


In [ ]:
# saving to csv
df11_filtered.to_csv('/content/drive/MyDrive/Uni/SocialMedSeminar/Daten/FINAL/tiktok_gpt_annotiert_climatechange_filtered.csv')
df22_filtered.to_csv('/content/drive/MyDrive/Uni/SocialMedSeminar/Daten/FINAL/tiktok_gpt_annotiert_climatecrisis_filtered.csv')
df33_filtered.to_csv('/content/drive/MyDrive/Uni/SocialMedSeminar/Daten/FINAL/tiktok_gpt_annotiert_klimakrise_filtered.csv')
df44_filtered.to_csv('/content/drive/MyDrive/Uni/SocialMedSeminar/Daten/FINAL/tiktok_gpt_annotiert_klimaschutz_filtered.csv')
df55_filtered_for_real.to_csv('/content/drive/MyDrive/Uni/SocialMedSeminar/Daten/FINAL/tiktok_gpt_annotiert_klimawandel_filtered.csv')
df66_filtered_for_real.to_csv('/content/drive/MyDrive/Uni/SocialMedSeminar/Daten/FINAL/tiktok_gpt_annotiert_savetheplanet_filtered.csv')

In [ ]:
# appending insta
insta_dfs = [df1_filtered, df2_filtered, df3_filtered, df4_filtered, df5_filtered_for_real, df6_filtered]

total_insta_df = pd.DataFrame()

for df in insta_dfs:
    total_insta_df = total_insta_df.append(df)

total_insta_df

In [ ]:
# appending tiktok
tiktok_dfs = [df11_filtered, df22_filtered, df33_filtered, df44_filtered, df55_filtered_for_real, df66_filtered_for_real]

total_tiktok_df = pd.DataFrame()

for df in tiktok_dfs:
    total_tiktok_df = total_tiktok_df.append(df)

total_tiktok_df

<ipython-input-65-7b84bde706d9>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_tiktok_df = total_tiktok_df.append(df)
<ipython-input-65-7b84bde706d9>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_tiktok_df = total_tiktok_df.append(df)
<ipython-input-65-7b84bde706d9>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_tiktok_df = total_tiktok_df.append(df)
<ipython-input-65-7b84bde706d9>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_tiktok_df = total_tiktok_df.append(df)
<ipython-input-65-7b84bde706d9>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version.

,Unnamed: 0.1,Unnamed: 0,id,thread_id,author,author_full,author_followers,author_likes,author_videos,author_avatar,...,image_path,Correlation_to_the_topic,Type_of_media_content,Impression_or_Sentiment,Animals,Nature,People,Natural_Disasters,Artifacts,Education_Facts_or_News
0,0,0,7172368921318657322,7172368921318657322,creativesociety.official,Creative Society project,226500,1555,1060,NaN,...,/content/drive/MyDrive/Uni/SocialMedSeminar/Da...,True,Photo,Negative,False,True,True,False,False,True
1,1,1,7256810993311812907,7256810993311812907,wffnews,WFF News,148500,4724,181,NaN,...,/content/drive/MyDrive/Uni/SocialMedSeminar/Da...,True,Photo,Negative,False,False,True,False,True,False
2,2,2,7224978138751618305,7224978138751618305,thethinkingreatape,TheThinkingreatApe,32400,15,337,NaN,...,/content/drive/MyDrive/Uni/SocialMedSeminar/Da...,False,Photo,Neutral,False,False,True,False,False,False
3,3,3,7291671021827116334,7291671021827116334,just_breathe_sunshine,Just_breathe,52,37,22,NaN,...,/content/drive/MyDrive/Uni/SocialMedSeminar/Da...,True,Photo,Negative,False,False,True,True,False,True
4,4,4,7252904683222895918,7252904683222895918,cbsnews,cbsnews,3600000,633,1669,NaN,...,/content/drive/MyDrive/Uni/SocialMedSeminar/Da...,True,Photo,Negative,False,False,True,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3098,3098,3098,7116909805238848773,7116909805238848773,holdcrofthandmades,Evie | Holdcroft Handmades,3540,2612,102,NaN,...,/content/drive/MyDrive/Uni/SocialMedSeminar/Da...,False,Photo,Positive,False,False,True,False,True,True
3099,3099,3099,7034644508419820846,7034644508419820846,rewildcarlyrose,carly rose 🌹,82200,318,168,NaN,...,/content/drive/MyDrive/Uni/SocialMedSeminar/Da...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3100,3100,3100,7126465295186742534,7126465295186742534,joo_su,Josu 🌎,1600000,22600,793,NaN,...,/content/drive/MyDrive/Uni/SocialMedSeminar/Da...,True,Photo,Neutral,False,True,True,False,True,False
3101,3101,3101,6877839838687595777,6877839838687595777,angryearth,ANGRY EARTH,852000,152,250,NaN,...,/content/drive/MyDrive/Uni/SocialMedSeminar/Da...,True,Drawing,Negative,True,False,False,False,False,True


In [ ]:
total_insta_df.to_csv('/content/drive/MyDrive/Uni/SocialMedSeminar/Daten/FINAL/insta_gpt_annotiert_total.csv')

In [ ]:
total_tiktok_df.to_csv('/content/drive/MyDrive/Uni/SocialMedSeminar/Daten/FINAL/tiktok_gpt_annotiert_total.csv')